# Collect Census Data in Python

Collect Census data with [Census API](https://www.census.gov/data/developers/data-sets.html) and store the information to a PostgreSQL database. 

## Install Python Packages  

- census: a Census API wrapper to call Census APIs
- us: a package praovidng US states antd territories names and FIPS codes
- psycopg2: PostgreSQL databases adapter to create tables, insert data, and execute queries

In [19]:
pip install census

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install us

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


## Secret Manager Function

In [13]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Connect to Database

In [14]:
import psycopg2

conn = psycopg2.connect(host=get_secret('postgresql2')['host'],
                      user=get_secret('postgresql2')['username'],
                      password = get_secret('postgresql2')['password'],
                      dbname=get_secret('postgresql2')['engine'])

cur = conn.cursor()

## Create Tables

In [15]:
sql = """
-- Create tables
CREATE TABLE IF NOT EXISTS name
(
    fips VARCHAR(4) NOT NULL UNIQUE,
    name VARCHAR(100) UNIQUE,
    PRIMARY KEY(fips)
);

CREATE TABLE IF NOT EXISTS population
(
    fips VARCHAR(4) NOT NULL,
    pop INTEGER NOT NULL,
    year INTEGER NOT NULL,
    PRIMARY KEY(fips, year)
);

CREATE TABLE IF NOT EXISTS income
(
    fips VARCHAR(4) NOT NULL,
    income INTEGER NOT NULL,
    year INTEGER NOT NULL,
    PRIMARY KEY(fips, year)
);


-- Create FKs
ALTER TABLE population
    ADD    FOREIGN KEY (fips)
    REFERENCES name(fips)
    MATCH SIMPLE
;
    
ALTER TABLE income
    ADD    FOREIGN KEY (fips)
    REFERENCES name(fips)
    MATCH SIMPLE
;

"""

In [16]:
cur.execute(sql)
conn.commit()

## Insert Census Data into Tables

Load Census API keys

In [17]:
from census import Census
from us import states


c = Census(get_secret('census')['api_key'])

Insert state names to the name table.

In [18]:
for state in states.STATES_AND_TERRITORIES:
    sql = f"""insert into name (name,fips) 
            values ('{state}','{state.fips}') """
    cur.execute(sql)
    
sql = f"""insert into name (name,fips) 
            values ('{states.DC.name}','{states.DC.fips}') """
cur.execute(sql)

conn.commit()

UniqueViolation: duplicate key value violates unique constraint "name_pkey"
DETAIL:  Key (fips)=(01) already exists.


Insert the population data to the population table.

In [ ]:
for year in range(2005,2020):
    print('collecting data in year of: ',year)
    for data in c.acs1.get('B01003_001E',{'for': 'state:*'},year=year):
        
        sql = f"""insert into population (fips,pop,year) 
            values ('{data['state']}',{data['B01003_001E']},{year}) """
#         print(sql)
        cur.execute(sql)
    conn.commit()

Insert the median household income into the income table.

In [ ]:
for year in range(2005,2020):
    print('collecting data in year of: ',year)
    for data in c.acs1.get('B19013_001E',{'for': 'state:*'},year=year):
        
        sql = f"""insert into income (fips,income,year) 
            values ('{data['state']}',{data['B19013_001E']},{year}) """
#         print(sql)
        cur.execute(sql)
    conn.commit()


## Close Dabase Connection

In [ ]:
cur.close()
conn.close()